# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [21]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [22]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX")

# Import Data

In [23]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [39]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    # YOUR CODE HERE!
    mean_units_yr = sfo_data.groupby(['year']).mean()
    mean_housing_units = mean_units_yr.plot.bar(y='housing_units', ylim=(370000,387500))
    return mean_housing_units

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    mean_units_yr = sfo_data.groupby(['year']).mean()
    mean_gross_rent = mean_units_yr.plot.line(y='gross_rent', title='Average Gross Rent per Year')
    return mean_gross_rent

def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    mean_units_yr = sfo_data.groupby(['year']).mean()
    mean_sales_price_per_sqft = mean_units_yr.plot.line(y='sale_price_sqr_foot', title='Average Sales Price per Year')
    return mean_sales_price_per_sqft

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    avg_neighborhood_prices = sfo_data.groupby(['year', 'neighborhood']).mean().reset_index()
    plot_neighboorhood_prices = avg_neighborhood_prices.hvplot.line(x='year', y='sale_price_sqr_foot', groupby='neighborhood', title='Average Sales Price per SQFT by Neighboorhood')
    return plot_neighboorhood_prices

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    # YOUR CODE HERE!
    expensive_neighborhood_prices = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False).head(10).reset_index()
    plot_expensive_neighboorhood_prices = expensive_neighborhood_prices.hvplot.bar(x='neighborhood', y='sale_price_sqr_foot', title='10 Most Expensive Neighborhoods')
    return plot_expensive_neighboorhood_prices

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    # YOUR CODE HERE!
    expensive_neighborhood_prices = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False).head(10).reset_index()
    parallel_coordinates = px.parallel_coordinates(expensive_neighborhood_prices, color='sale_price_sqr_foot')
    return parallel_coordinates

def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    expensive_neighborhood_prices = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False).head(10).reset_index()
    parallel_categories = px.parallel_categories(
    expensive_neighborhood_prices,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    labels={
        "neighboorhood": "Neighborhood",
        "sale_price_sqr_foot": "Sales Price by SQFT",
        "housing_units": "Housing Units",
        "gross_rent": "Gross Rent"
        },
    )
    return parallel_categories

def neighborhood_map():
    """Neighborhood Map"""
    
    # YOUR CODE HERE!
    file_path = Path("Data/neighborhoods_coordinates.csv")
    df_neighborhood_locations = pd.read_csv(file_path)
    mean_prices_neighborhood = sfo_data.groupby(['neighborhood']).mean().reset_index()
    mean_prices_final = mean_prices_neighborhood.join(df_neighborhood_locations.set_index('Neighborhood'), on='neighborhood', how='inner')
    map = px.scatter_mapbox(
    mean_prices_final,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    zoom=4
    )
    return map


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [45]:
# YOUR CODE HERE!
pandas_plots = pn.Column("## Pandas Plots", housing_units_per_year(), average_gross_rent(), average_sales_price())
hvplot_plots = pn.Column("## Hvplot Plots", top_most_expensive_neighborhoods(), average_price_by_neighborhood())
plotly_plots = pn.Column("## Plotly Plots", parallel_coordinates(), parallel_categories())
mapbox_plots = pn.Column("## Mapbox Plots", neighborhood_map())
dashboard = pn.Tabs(
    ("Pandas Plots", pandas_plots),
    ("Hvplot Plots", hvplot_plots),
    ("Plotly Plots", plotly_plots),
    ("Mapbox Plots", mapbox_plots)
)
dashboard

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [1] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)

## Serve the Panel Dashboard

In [43]:
dashboard.servable()

Tabs(active=2)
    [0] Column
        [0] Markdown(str)
        [1] Str(AxesSubplot)
        [2] Str(AxesSubplot)
        [3] Str(AxesSubplot)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)